In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Load data

In [6]:
DATA_FOLDER = '../readonly/final_project_data/'

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [12]:
sales["sales"] = sales["item_price"] * sales["item_cnt_day"]
sales["month"] = sales["date"].map(lambda x: ".".join(reversed(x.split(".")[1:])))
salesByShopItemMonth = sales.groupby(["shop_id", "item_id", "month"])["sales"].count().reset_index()
salesByShopItemMonth.head()

,shop_id,item_id,month,sales
0,0,30,2013.02,9
1,0,31,2013.02,7
2,0,32,2013.01,4
3,0,32,2013.02,7
4,0,33,2013.01,3


In [15]:
monthlySalesByShopItem = salesByShopItemMonth.groupby(["shop_id", "item_id"])["sales"].mean().reset_index()
monthlySalesByShopItem.head()

,shop_id,item_id,sales
0,0,30,9.0
1,0,31,7.0
2,0,32,5.5
3,0,33,3.0
4,0,35,6.0


In [16]:
pred = pd.merge(test, monthlySalesByShopItem, on=["shop_id","item_id"], how = "left")
pred.head()

,ID,shop_id,item_id,sales
0,0,5,5037,1.444444
1,1,5,5320,NaN
2,2,5,5233,1.800000
3,3,5,5232,1.000000
4,4,5,5268,NaN


In [17]:
pred["item_cnt_month"] = pred["sales"]
pred["item_cnt_month"].fillna(0, inplace=True)
pred["item_cnt_month"] = pred["item_cnt_month"].map(lambda x: min(max(0, x), 20))

In [18]:
pred[["ID", "item_cnt_month"]].to_csv("submission_constant.csv", index=False)

In [21]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
